In [15]:
import os
import gc
import sys
import torch
import psutil
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn import metrics
from collections import Counter
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import models, set_image_backend
import data_utils
import train_utils
import model_utils
import argparse


def make_data_frame_summary(labels_store,jpg_to_samples,outputs):
    df = pd.DataFrame({'labels':labels_store,'slide_id':jpg_to_samples})
    for i in range(5):
        df[str(i)] = np.concatenate(outputs[i])
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none')
    slide_level = df.groupby('slide_id').mean()
    loss = torch.zeros((len(slide_level),5))
    for i in range(5):
        labels = torch.tensor(slide_level['labels'].values).float()
        logits = torch.tensor(slide_level[str(i)].values)
        loss[:,i] = criterion(logits,labels)
    return slide_level

In [2]:
state_dict_file = '/n/tcga_models/resnet18_WGD_all_10x.pt'
input_size = 2048
hidden_size = 512
output_size = 1

# initialize trained resnet
resnet = models.resnet18(pretrained=False)
resnet.fc = nn.Linear(2048, output_size, bias=True)
saved_state = torch.load(state_dict_file, map_location=lambda storage, loc: storage)
resnet.load_state_dict(saved_state)
device = torch.device('cuda',1)
resnet.cuda(device=device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [3]:

for param in resnet.parameters():
    param.requires_grad = False
    
for param in resnet.fc.parameters(): 
    param.requires_grad = True

In [4]:
val_cancers = ['UCEC', 'LIHC_10x', 'KIRC_10x']
root_dir = '/n/mounted-data-drive/'
magnification = '10.0'
criterion=nn.BCEWithLogitsLoss()

In [5]:
pickle_file = '/home/sxchao/MSI_prediction/tcga_project/tcga_wgd_sa_all.pkl'
batch_all, _, sa_val1, sa_trains, sa_val2 = data_utils.load_COAD_train_val_sa_pickle(pickle_file=pickle_file,
                                                                               return_all_cancers=True, 
                                                                               split_in_two=True)
val_transform = train_utils.transform_validation

In [6]:
val_sets = []
sa_vals = [dict(sa_val1[idx], **sa_val2[idx]) for idx,_ in enumerate(sa_val1)]


In [7]:
for j in range(len(val_cancers)):
    val_set = data_utils.TCGADataset_tiles(sa_vals[batch_all.index(val_cancers[j])], 
                                           root_dir + val_cancers[j] + '/', 
                                           transform=val_transform, 
                                           magnification=magnification, 
                                           batch_type='tile', 
                                           return_jpg_to_sample=True)
    val_sets.append(val_set)

In [8]:
val_loaders = [torch.utils.data.DataLoader(val_set, 
                                            batch_size=400, 
                                            shuffle=True, 
                                            num_workers=12, 
                                            pin_memory=True,drop_last=True) for val_set in val_sets]

optimizer = torch.optim.Adam(resnet.fc.parameters(), lr = 1e-3) 

In [19]:
dfs = dict()
for jx,val_loader in enumerate(val_loaders):
    if jx >0:
        continue
    print(len(val_loader))
    losses = np.zeros([10,len(val_loader)])
    original_weights = [p.detach().clone() for p in resnet.fc.parameters()]
    jpg_to_samples = []
    labels_store = []
    outputs = dict()
    for step in range(5):
        outputs[step] = []

    for step, (batch,labels,jpg_to_sample) in enumerate(val_loader):
        jpg_to_samples.extend(jpg_to_sample[200:].cpu().numpy())
        labels_store.extend(labels[200:].cpu().numpy())
        resnet.fc.weight.data = original_weights[0].clone()
        resnet.fc.bias.data = original_weights[1].clone()
        inputs, labels = batch.cuda(device=device), labels.cuda(device=device).view(-1,1).float()
        inputs_a = inputs[:200,:,:,:]
        inputs_b = inputs[200:,:,:,:]
        output_b = resnet(inputs_b)
        outputs[0].append(output_b.detach().cpu().numpy())

        loss = criterion(output_b, labels[200:,:]) 
        losses[0,step] = loss.detach().cpu().numpy()
        print(step,end=' ')

        for ix in range(1,5):

            output_a = resnet(inputs_a)
            loss = criterion(output_a, labels[:200,:])
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()


            output_b = resnet(inputs_b)
            outputs[ix].append(output_b.detach().cpu().numpy())
            loss = criterion(output_b, labels[200:,:])
            losses[ix,step] = loss.detach().cpu().numpy()
    dfs[jx] = make_data_frame_summary(labels_store,jpg_to_samples,outputs)

988
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 27

In [26]:
[dfs[k].to_csv(val_cancers[ix]+'_results_baseline_slidelevel_val.txt',sep='\t',index_label='slide_id') for ix,k in enumerate(dfs.keys())]

[None, None, None]

In [24]:
val_cancers

['UCEC', 'LIHC_10x', 'KIRC_10x']

In [ ]:
from sklearn import metrics
import scipy
fpr, tpr, thresholds = metrics.roc_curve(np.array(slide_level['labels']), scipy.special.expit(np.array(slide_level['1'])))
roc_auc = metrics.auc(fpr, tpr)


In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Slide-level receiver operating characteristic (WGD)')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import scipy
roc_auc = []
for i in range(5):
    fpr, tpr, thresholds = metrics.roc_curve(np.array(slide_level['labels']), scipy.special.expit(np.array(slide_level[str(i)])))
    roc_auc.append( metrics.auc(fpr, tpr))


In [ ]:
plt.plot(roc_auc,'b.')

In [ ]:
slide_level.to_csv('slide_level_resnet_baseline_LIHC.txt',index_label='slide_id',sep='\t')